# Part 2 -- Pre-Processing and Text Cleaning

Next we're going to do some basic pre-processing, like converting the text to lower case and deleting URLs. We don't need to remove stopwords such as "the" and "a", which will be done when we pass our data through a NLTK model later. 

**Load lib codes**

In [2]:
from os import chdir
chdir('/home/jovyan/work/Analyzing_Unstructured_Data_for_Finance/Analyzing_Unstructured_Data_for_Finance/')

from lib import *
# suppress_warnings()

In [5]:
!pip install pymongo
import pymongo

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# Identify port for better security of MongoDB
cli = pymongo.MongoClient(host='52.27.11.214', port=27016)

In [7]:
completed_collection = cli.twitter_db.completed_collection
cli.twitter_db.collection_names()

['completed_collection', 'task_collection']

**Clean data**

In [8]:
# Get data out of our MongoDB collection
tweets_list = [document for document in completed_collection.find()]
tweets_df = pd.DataFrame(tweets_list)
tweets_df.head(2)

,_id,text,timestamp,username
0,593deb7057bbd4047664223e,"On this National Gun Violence Awareness Day, l...",2017-06-02 17:35:54,BarackObama
1,593deb7057bbd4047664223f,Forever grateful for the service and sacrifice...,2017-05-29 13:09:16,BarackObama


In [9]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94856 entries, 0 to 94855
Data columns (total 4 columns):
_id          94856 non-null object
text         94856 non-null object
timestamp    94856 non-null datetime64[ns]
username     94856 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 2.9+ MB


In [10]:
# Make our text into a list (instead of a series) so we can apply functions to it
tweets_text = []
for i in range(len(tweets_df)):
    tweets_text.append(tweets_list[i]['text'])

In [11]:
import re

In [14]:
def clean_text(text):
    text = re.sub('(http\S+)', '', text)
    text = re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', text)
    text = re.sub('[\W]', ' ', text)
    text = re.sub('(\@.*?(?=\s))', ' ', text)
    text = re.sub('[\W, \d]', ' ', text)
    text = re.sub('(\s[a-z]{1}\s)', ' ', text)
                  
    text = re.sub('([\s]{2,})', ' ', text)
    text = text.lower()
    return text

In [15]:
cleaned_text = []
for t in tweets_text:
    cleaned_text.append(clean_text(t))

In [16]:
cleaned_text[0]

'on this national gun violence awareness day let your voice be heard and show your commitment to reducing gun viole '

In [17]:
tweets_df['cleaned_text'] = cleaned_text

In [18]:
tweets_df['Date'] = tweets_df['timestamp'].dt.date
tweets_df['Date'] = pd.to_datetime(tweets_df['Date'])

In [19]:
tweets_df.sample(5)

,_id,text,timestamp,username,cleaned_text,Date
21825,593debb257bbd40476647786,"Tune in to Microsoft's E3 ""Project Scorpio"" li...",2017-06-11 17:56:34,TechCrunch,tune in to microsoft e project scorpio live st...,2017-06-11
71783,593dec4057bbd40476653abb,RT @DamatoK: Here's a straightforward pencil-a...,2017-04-13 13:13:23,MONEY,rt damatok here a straightforward pencil and p...,2017-04-13
79534,593dec5c57bbd40476655905,"SocGen Angered By ""Stupidly Strong"" Correlatio...",2017-06-06 14:00:48,zerohedge,socgen angered by stupidly strong correlation ...,2017-06-06
22771,593debb457bbd40476647b38,Pokémon Go reveals sponsors like McDonald's pa...,2017-05-31 16:27:30,TechCrunch,pokémon go reveals sponsors like mcdonald pay ...,2017-05-31
443,593deb7157bbd404766423f9,#OrlandoUnited https://t.co/ixCxMZzGH2,2016-06-13 16:52:45,BarackObama,orlandounited,2016-06-13


In [20]:
tweets_df['Date'].dtype

dtype('<M8[ns]')

In [21]:
tweets_df.shape

(94856, 6)

In [22]:
joblib.dump(tweets_df, '../Analyzing_Unstructured_Data_for_Finance/data_cleaned/b1.tweets_df.pickle')

['../Analyzing_Unstructured_Data_for_Finance/data_cleaned/b1.tweets_df.pickle']

In [23]:
# Use the dates to determine how the date range we want to pull stock data from
print(min(tweets_df['Date']))
print(max(tweets_df['Date']))

2008-09-10 00:00:00
2017-06-12 00:00:00
